In [ ]:
# TensorFlow and tf.keras
import math
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras


#tf version should be 2.5 or higher
tf.__version__

'2.6.0'

In [143]:
train=pd.read_csv('sign_mnist_train.csv')
test=pd.read_csv('sign_mnist_test.csv')

In [144]:
train_images=train.values[0:,1:]

test_images=test.values[0:,1:]

In [145]:
labels = train.values[0:,0]




labels_test = test.values[0:,0]


In [147]:
train_images=train_images/255
train_images=train_images.reshape((27455,28,28,1))

test_images=test_images/255

test_images=test_images.reshape((7172, 28,28,1))


In [177]:
#setup model
model = keras.Sequential([
   
    
    keras.layers.Conv2D(kernel_size=3, filters=32, use_bias=False, padding='same', input_shape=(28, 28 , 1)),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
    
    
    keras.layers.Conv2D(kernel_size=6, filters=64, use_bias=False, padding='same', strides=2),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
   
    
    keras.layers.Conv2D(kernel_size=6, filters=64, use_bias=False, padding='same', strides=2),
    keras.layers.BatchNormalization(center=True, scale=False),
    keras.layers.Activation('relu'),
    
        
    
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
   
    
    keras.layers.Flatten(),
    

    
    keras.layers.Dense(64, use_bias=False),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(25, activation = tf.nn.softmax)
])

In [178]:
#compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [179]:
def lr_decay(epoch):
    return 0.01*math.pow(0.6, epoch)

lr_decay_callback=tf.keras.callbacks.LearningRateScheduler(lr_decay)

In [180]:
#train model
model.fit(train_images, labels, validation_split=0.1, epochs=6, batch_size=64, callbacks=[lr_decay_callback])

Epoch 1/6
387/387 [==============================] - 12s 30ms/step - loss: 1.5094 - accuracy: 0.4912 - val_loss: 3.1681 - val_accuracy: 0.1173
Epoch 2/6
387/387 [==============================] - 11s 29ms/step - loss: 0.2294 - accuracy: 0.9198 - val_loss: 0.0590 - val_accuracy: 0.9876
Epoch 3/6
387/387 [==============================] - 12s 32ms/step - loss: 0.1062 - accuracy: 0.9644 - val_loss: 0.0245 - val_accuracy: 0.9927
Epoch 4/6
387/387 [==============================] - 11s 29ms/step - loss: 0.0834 - accuracy: 0.9720 - val_loss: 0.0029 - val_accuracy: 0.9993
Epoch 5/6
387/387 [==============================] - 11s 29ms/step - loss: 0.0622 - accuracy: 0.9773 - val_loss: 7.7768e-04 - val_accuracy: 1.0000
Epoch 6/6
387/387 [==============================] - 11s 29ms/step - loss: 0.0500 - accuracy: 0.9819 - val_loss: 4.8021e-04 - val_accuracy: 1.0000


In [181]:
#evaluate
test_loss, test_acc = model.evaluate(test_images,  labels_test)
print('\nTest accuracy:', test_acc)

225/225 [==============================] - 1s 5ms/step - loss: 0.2961 - accuracy: 0.9320

Test accuracy: 0.9319576025009155


In [182]:
_, baseline_model_accuracy = model.evaluate(
    test_images, labels_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9319576025009155
Saving model to:  /tmp/tmpe6a_rpe7.h5


In [183]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}


clustered_model = cluster_weights(model, **clustering_params)


opt = tf.keras.optimizers.Adam(learning_rate=0.01)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cluster_conv2d_27 (ClusterWe (None, 28, 28, 32)        592       
_________________________________________________________________
cluster_batch_normalization_ (None, 28, 28, 32)        96        
_________________________________________________________________
cluster_activation_46 (Clust (None, 28, 28, 32)        0         
_________________________________________________________________
cluster_max_pooling2d_27 (Cl (None, 14, 14, 32)        0         
_________________________________________________________________
cluster_conv2d_28 (ClusterWe (None, 7, 7, 64)          147472    
_________________________________________________________________
cluster_batch_normalization_ (None, 7, 7, 64)          192       
_________________________________________________________________
cluster_activation_47 (Clust (None, 7, 7, 64)         

In [186]:
# Fine-tune model
clustered_model.fit(
  train_images,
  labels,
  batch_size=64,
  epochs=6,
  validation_split=0.1,
  callbacks=[lr_decay_callback])

Epoch 1/6
387/387 [==============================] - 11s 28ms/step - loss: 2.4203 - accuracy: 0.8655 - val_loss: 2.4614 - val_accuracy: 0.8230
Epoch 2/6
387/387 [==============================] - 11s 29ms/step - loss: 2.3390 - accuracy: 0.9466 - val_loss: 2.3466 - val_accuracy: 0.9381
Epoch 3/6
387/387 [==============================] - 11s 29ms/step - loss: 2.3069 - accuracy: 0.9784 - val_loss: 2.2901 - val_accuracy: 0.9953
Epoch 4/6
387/387 [==============================] - 11s 29ms/step - loss: 2.2896 - accuracy: 0.9958 - val_loss: 2.2873 - val_accuracy: 0.9978
Epoch 5/6
387/387 [==============================] - 11s 28ms/step - loss: 2.2865 - accuracy: 0.9989 - val_loss: 2.2861 - val_accuracy: 0.9993
Epoch 6/6
387/387 [==============================] - 11s 28ms/step - loss: 2.2855 - accuracy: 0.9998 - val_loss: 2.2853 - val_accuracy: 1.0000


In [187]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, labels_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.9319576025009155
Clustered test accuracy: 0.9277746677398682


In [188]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Saving clustered model to:  /tmp/tmp5gzdl6og.h5


In [189]:
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmpjk6w5k4y/assets


INFO:tensorflow:Assets written to: /tmp/tmpjk6w5k4y/assets


Saved clustered TFLite model to: /tmp/clustered_mnist.tflite


In [190]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [191]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

Size of gzipped baseline Keras model: 847758.00 bytes
Size of gzipped clustered Keras model: 97950.00 bytes
Size of gzipped clustered TFlite model: 105926.00 bytes


In [192]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_clustered_tflite_file, 'wb') as f:
  f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', quantized_and_clustered_tflite_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmp28y67_0l/assets


INFO:tensorflow:Assets written to: /tmp/tmp28y67_0l/assets


Saved quantized and clustered TFLite model to: /tmp/tmpsz85zqos.tflite
Size of gzipped baseline Keras model: 847758.00 bytes
Size of gzipped clustered and quantized TFlite model: 83309.00 bytes


In [196]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]


  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    .
    interpreter.invoke()

  
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
 
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == labels_test).mean()
  return accuracy

In [197]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Clustered and quantized TFLite test_accuracy:', test_accuracy)
print('Clustered TF test accuracy:', clustered_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.


Clustered and quantized TFLite test_accuracy: 0.9244283324037925
Clustered TF test accuracy: 0.9277746677398682


In [198]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))
print("Size of gzipped quantized and clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

Size of gzipped baseline Keras model: 847758.00 bytes
Size of gzipped clustered Keras model: 97950.00 bytes
Size of gzipped clustered TFlite model: 105926.00 bytes
Size of gzipped quantized and clustered TFlite model: 83309.00 bytes
